In [1]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=ee108f3e23bfff4b4c9908b58e813fda5f7c4a2d3d073325509524e60e4f409b
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
import numpy as np
import pandas as pd
import pandas_ta as ta

In [3]:
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve

# Download data
print(f'Downloading OIH_adjusted.txt...')
urlretrieve('http://api.kibot.com/?action=history&symbol=OIH&interval=1&unadjusted=0&bp=1&user=guest', 'OIH_adjusted.txt')

# Read data and assign names to the columns
df = pd.read_csv('OIH_adjusted.txt')
df.columns = ['date','time','open','high','low','close','volume']

# Combine date and time in the date column
df['date'] = df['date'] + ' ' + df['time']
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M')
df = df[['date','open','high','low','close','volume']]

# Sort by date and assign the date as index
df = df.sort_values('date').reset_index(drop=True).set_index('date')

# Convert the data to different timeframes & save them for future uses
AGGREGATION = {'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}
TIMEFRAMES = ['5T', '15T', '1H', '1D']

for timeframe in TIMEFRAMES:
    print(f'Converting & Saving {timeframe} Data...')
    df = df.resample(timeframe).agg(AGGREGATION).dropna()
    df.to_csv(f'OIH_{timeframe}.csv.gz', compression='gzip')


Converting & Saving 5T Data...
Converting & Saving 15T Data...
Converting & Saving 1H Data...
Converting & Saving 1D Data...


In [4]:
TIMEFRAMES = ['5T', '15T', '1H', '1D']

In [5]:
# Using this method, you can obtain buy and sell signals determined by the selected strategy.
# The resulting signals are represented as a series of numerical values:
#   '1' indicating a buy signal,
#   '0' indicating a hold signal, and
#   '-1' indicating a sell signal
def get_signals(df):

    pd.options.mode.chained_assignment = None
    
    df.ta.bbands(close=df['close'], length=20, append=True)    
    df = df.dropna()
    
    df['high_limit'] = df['BBU_20_2.0'] + (df['BBU_20_2.0'] - df['BBL_20_2.0']) / 2
    df['low_limit'] = df['BBL_20_2.0'] - (df['BBU_20_2.0'] - df['BBL_20_2.0']) / 2
    df['close_percentage'] = np.clip((df['close'] - df['low_limit']) / (df['high_limit'] - df['low_limit']), 0, 1)
    df['volatility'] = df['BBU_20_2.0'] / df['BBL_20_2.0'] - 1
    min_volatility = df['volatility'].mean() - df['volatility'].std()
    
    # Buy Signals
    df['signal'] = np.where((df['volatility'] > min_volatility) & (df['close_percentage'] < 0.25), 1, 0)
    
    # Sell Signals
    df['signal'] = np.where((df['close_percentage'] > 0.75), -1, df['signal'])
    
    return df['signal']

# Using this method, you can visualize the results of a simulated long position strategy.
# Note that it assumes the purchase of one share per transaction and does not account for any fees.
def show_stategy_result(timeframe, df):

    waiting_for_close = False
    open_price = 0

    profit = 0.0
    wins = 0
    losses = 0

    for i in range(len(df)):

        signal = df.iloc[i]['signal']

        if signal == 1 and not waiting_for_close:
            waiting_for_close = True
            open_price = df.iloc[i]['close']

        elif signal == -1 and waiting_for_close:
            waiting_for_close = False
            close_price = df.iloc[i]['close']

            profit += close_price - open_price
            wins = wins + (1 if (close_price - open_price) > 0 else 0)
            losses = losses + (1 if (close_price - open_price) < 0 else 0)

    print(f' Result for timeframe {timeframe} '.center(60, '*'))
    print(f'* Profit/Loss: {profit:.2f}')
    print(f"* Wins: {wins} - Losses: {losses}")
    print(f"* Win Rate: {100 * (wins/(wins + losses)):6.2f}%")


In [6]:
# Iterate over each timeframe, apply the strategy and show the result
for timeframe in TIMEFRAMES:

    # Read the data
    df = pd.read_csv(f'OIH_{timeframe}.csv.gz', compression='gzip')

    # Add the signals to each row
    df['signal'] = get_signals(df)

    # Get the result of the strategy
    show_stategy_result(timeframe, df)

***************** Result for timeframe 5T ******************
* Profit/Loss: 2403.68
* Wins: 3379 - Losses: 1851
* Win Rate:  64.61%
***************** Result for timeframe 15T *****************
* Profit/Loss: 2220.58
* Wins: 1369 - Losses: 726
* Win Rate:  65.35%
***************** Result for timeframe 1H ******************
* Profit/Loss: 251.15
* Wins: 417 - Losses: 203
* Win Rate:  67.26%
***************** Result for timeframe 1D ******************
* Profit/Loss: -122.69
* Wins: 43 - Losses: 19
* Win Rate:  69.35%
